# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 30 2022 2:21PM,254078,12,20230103-AZFL,SugarBear,Executing
1,Dec 30 2022 2:03PM,254077,12,8968843,"SD Head USA, LLC",Released
2,Dec 30 2022 1:50PM,254076,19,ACG-2102492318,ACG North America LLC,Released
3,Dec 30 2022 12:21PM,254062,16,8965070,Sartorius Bioprocess Solutions,Released
4,Dec 30 2022 11:32AM,254057,19,8968258,"Innogenix, LLC",Released
5,Dec 30 2022 11:32AM,254057,19,8968259,"Innogenix, LLC",Released
6,Dec 30 2022 11:32AM,254057,19,8968260,"Innogenix, LLC",Released
7,Dec 30 2022 11:32AM,254057,19,8968261,"Innogenix, LLC",Released
8,Dec 30 2022 11:32AM,254057,19,8968262,"Innogenix, LLC",Released
9,Dec 30 2022 11:32AM,254057,19,8968263,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,254057,Released,23
7,254062,Released,1
8,254076,Released,1
9,254077,Released,1
10,254078,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254057,NaN,23.0
254062,NaN,1.0
254076,NaN,1.0
254077,NaN,1.0
254078,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254027,0.0,1.0
254042,0.0,1.0
254045,0.0,1.0
254046,0.0,1.0
254048,6.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254027,0,1
254042,0,1
254045,0,1
254046,0,1
254048,6,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254027,0,1
1,254042,0,1
2,254045,0,1
3,254046,0,1
4,254048,6,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254027,,1
1,254042,,1
2,254045,,1
3,254046,,1
4,254048,6,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 30 2022 2:21PM,254078,12,SugarBear
1,Dec 30 2022 2:03PM,254077,12,"SD Head USA, LLC"
2,Dec 30 2022 1:50PM,254076,19,ACG North America LLC
3,Dec 30 2022 12:21PM,254062,16,Sartorius Bioprocess Solutions
4,Dec 30 2022 11:32AM,254057,19,"Innogenix, LLC"
27,Dec 30 2022 10:06AM,254048,10,Bio-PRF
37,Dec 30 2022 9:13AM,254046,12,Livs Products
38,Dec 30 2022 9:08AM,254045,10,SugarBear
39,Dec 30 2022 9:01AM,254042,16,Sartorius Stedim Filters Inc.
40,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 30 2022 2:21PM,254078,12,SugarBear,1,
1,Dec 30 2022 2:03PM,254077,12,"SD Head USA, LLC",,1
2,Dec 30 2022 1:50PM,254076,19,ACG North America LLC,,1
3,Dec 30 2022 12:21PM,254062,16,Sartorius Bioprocess Solutions,,1
4,Dec 30 2022 11:32AM,254057,19,"Innogenix, LLC",,23
5,Dec 30 2022 10:06AM,254048,10,Bio-PRF,6,4
6,Dec 30 2022 9:13AM,254046,12,Livs Products,,1
7,Dec 30 2022 9:08AM,254045,10,SugarBear,,1
8,Dec 30 2022 9:01AM,254042,16,Sartorius Stedim Filters Inc.,,1
9,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 2:21PM,254078,12,SugarBear,,1
1,Dec 30 2022 2:03PM,254077,12,"SD Head USA, LLC",1,
2,Dec 30 2022 1:50PM,254076,19,ACG North America LLC,1,
3,Dec 30 2022 12:21PM,254062,16,Sartorius Bioprocess Solutions,1,
4,Dec 30 2022 11:32AM,254057,19,"Innogenix, LLC",23,
5,Dec 30 2022 10:06AM,254048,10,Bio-PRF,4,6
6,Dec 30 2022 9:13AM,254046,12,Livs Products,1,
7,Dec 30 2022 9:08AM,254045,10,SugarBear,1,
8,Dec 30 2022 9:01AM,254042,16,Sartorius Stedim Filters Inc.,1,
9,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 2:21PM,254078,12,SugarBear,,1
1,Dec 30 2022 2:03PM,254077,12,"SD Head USA, LLC",1,
2,Dec 30 2022 1:50PM,254076,19,ACG North America LLC,1,
3,Dec 30 2022 12:21PM,254062,16,Sartorius Bioprocess Solutions,1,
4,Dec 30 2022 11:32AM,254057,19,"Innogenix, LLC",23,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 2:21PM,254078,12,SugarBear,NaN,1.0
1,Dec 30 2022 2:03PM,254077,12,"SD Head USA, LLC",1.0,NaN
2,Dec 30 2022 1:50PM,254076,19,ACG North America LLC,1.0,NaN
3,Dec 30 2022 12:21PM,254062,16,Sartorius Bioprocess Solutions,1.0,NaN
4,Dec 30 2022 11:32AM,254057,19,"Innogenix, LLC",23.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 2:21PM,254078,12,SugarBear,0.0,1.0
1,Dec 30 2022 2:03PM,254077,12,"SD Head USA, LLC",1.0,0.0
2,Dec 30 2022 1:50PM,254076,19,ACG North America LLC,1.0,0.0
3,Dec 30 2022 12:21PM,254062,16,Sartorius Bioprocess Solutions,1.0,0.0
4,Dec 30 2022 11:32AM,254057,19,"Innogenix, LLC",23.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,762120,6.0,6.0
12,762201,2.0,1.0
16,508104,2.0,0.0
19,508133,24.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,762120,6.0,6.0
1,12,762201,2.0,1.0
2,16,508104,2.0,0.0
3,19,508133,24.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,6.0,6.0
1,12,2.0,1.0
2,16,2.0,0.0
3,19,24.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,6.0
1,12,Released,2.0
2,16,Released,2.0
3,19,Released,24.0
4,10,Executing,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,6.0,1.0,0.0,0.0
Released,6.0,2.0,2.0,24.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,6.0,1.0,0.0,0.0
1,Released,6.0,2.0,2.0,24.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,6.0,1.0,0.0,0.0
1,Released,6.0,2.0,2.0,24.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()